In [1]:
import pandas as pd
import numpy as np

def blight_model():
    
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_curve, auc, roc_auc_score
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

    # List the variables(Numeric and categorical) to be used
    
    use_cols_train = ['ticket_id','agency_name', 'inspector_name' ,'state', 'fine_amount','admin_fee','late_fee', 
                  'zip_code','violation_code', 'judgment_amount', 'compliance']
    use_cols_test = ['ticket_id','agency_name', 'inspector_name' ,'state', 'fine_amount','admin_fee' ,'late_fee', 
                  'zip_code','violation_code', 'judgment_amount']
    
    # Read training and testing data
    
    df_train  = pd.read_csv('train.csv', encoding = "ISO-8859-1", low_memory = False, 
                        usecols = use_cols_train)
    df_train = df_train[~(df_train['compliance'].isnull())]
    df_test = pd.read_csv('test.csv', encoding = "ISO-8859-1", low_memory = False, 
                        usecols = use_cols_test)
    
    # Read Addresses and Langitude- Longitude datasets. Merge them in 1 dataset with training and testing datasets

    df_addresses = pd.read_csv('addresses.csv')
    df_latlons =  pd.read_csv('latlons.csv')
    df_address_latlon = pd.merge(df_addresses, df_latlons, on = 'address', how = 'inner')
    df_train_temp = pd.merge(df_train, df_address_latlon, how = 'inner', on = 'ticket_id')
    df_test_temp = pd.merge(df_test, df_address_latlon, how = 'inner', on = 'ticket_id')
    
    
    # Fill NaN(NULL) values with random number

    df_train_temp.fillna(-100000, axis = 1, inplace = True)
    df_test_temp.fillna(-100000, axis = 1, inplace = True)

    
    # Prepare training and testing dataset. Set index to ticket_id

    X_col_list = ['ticket_id' ,'fine_amount' ,'agency_name','inspector_name' , 'admin_fee','late_fee','judgment_amount', 'lat', 'lon']
    
    X = df_train_temp.loc[:,X_col_list]
    y = df_train_temp.loc[:,'compliance']
    X.set_index('ticket_id', inplace = True)

    X_test_data = df_test_temp.loc[:,X_col_list]
    X_test_data.set_index('ticket_id', inplace = True )
    
    
        # Convert Categorical to numeric for Training Data

    for column in X.columns:
        text_digits_val = {}

        if X[column].dtypes not in ('int64','float64'):
            column_contents = list(X[column].unique()).astype(str)
            
            column_contents.sort()
            i = 0
            for content in column_contents:
                text_digits_val[content] = i
                i+=1
            X[column] = X[column].map(text_digits_val)
        
        
    # Convert Categorical to numeric for Testing Data

    for column in X_test_data.columns:
        text_digits_val = {}


        if X_test_data[column].dtypes not in ('int64','float64'):
            column_contents = list(X_test_data[column].unique())
            column_contents.sort()
            i = 0
            for content in column_contents:
                text_digits_val[content] = i
                i+=1
            X_test_data[column] = X_test_data[column].map(text_digits_val)
    


    # Using RandomForestClassifier as training model with tuned parameters
    
    RFC = RandomForestClassifier(n_estimators = 200,n_jobs = -1,  min_samples_leaf = 10, random_state = 1).fit(X, y)
    probs_test_data = RFC.predict_proba(X_test_data)
    return pd.Series(data=probs_test_data[:,1], index=X_test_data.index, dtype='float32')

blight_model()

ValueError: Usecols do not match names.

In [ ]:
print('he')